# Brasileirão Série A: O caminho dos campeões
Por [Daniel Soares](https://www.linkedin.com/in/daniel-soares-ti/)


<div style="display:inline-block;vertical-align:top;">
<h2>Sumário:</h2>
<ul>
    <li><a href="#introducao">Introdução </a></li>
    <li><a href="#responsabilidade">Reponsabilidade de cada Cargo</a></li>
     <li><a href="#tecnicas_metodos">Técnicas e Métodos</a></li>
    <li><a href="#ferramentas_servicos">Ferramentas e Serviços</a></li>
    <li><a href="#nivel_ensino">Nível de Ensino</a></li>
    <li><a href="#faixa_salarial">Faixal Salárial</a></li>
    <li><a href="#conclusao">Conclusão</a></li>
     <li><a href="#referencias">Referências</a></li>
</ul>
</div>
<div style="display:inline-block; margin-left:50px; margin-top:15px">
<img src="https://media.giphy.com/media/xUOxf1JRLpx5wy7UyY/giphy.gif" style="height: 250px; width:400px;"/>
    <div style="text-align:center; font-size:12px"><i>Fonte: <a href="https://media.giphy.com/media/xUOxf1JRLpx5wy7UyY/giphy.gif">Giphy</a></i></div>
</div>

--------


In [1]:
import pandas as pd
import plotly.express as px

In [2]:
# Lendo os dados
games_data = pd.read_csv('../data/campeonato-brasileiro-full.csv')

<a id='introducao'></a>
# Introdução
"O Brasileirão é o campeonato mais difícil do mundo". Essa é uma frase que você 
provavelmente já escutou na rua, e que vem provando ser verdadeira. A Federação 
Internacional de História e Estatísticas do Futebol (IFFHS), organização 
reconhecida pela FIFA, [publicou](https://www.iffhs.com/posts/2483) no início de 
2023 um ranking de campeonatos nacionais, e levando em conta fatores como força 
dos concorrentes e conquistas dos times na temporada, coroou o Brasileirão como 
o mais difícil do mundo pelo segundo ano consecutivo.

Diante disso, chegamos a uma pergunta: Qual o caminho dos campeões? O que um time
precisa para conquistar o atual campeonato mais difícil do planeta?

<div class="alert alert-warning" role="alert">
    <b>Aviso!</b> Foi desconsiderado o Campeonato Brasileiro de 2020.
</div>

In [3]:
# Para evitar problemas quando for plotar os gráficos, vou alterar o dtype
# das colunas que necessitam
games_data['data'] = pd.to_datetime(games_data['data'], dayfirst=True)
games_data['hora'] = pd.to_datetime(games_data['hora'], format='%H:%M').dt.time

# Para separar os campeonatos por ano, irei criar uma coluna com o ano 
# de cada jogo
games_data['ano'] = games_data['data'].dt.year

In [12]:
# Verificando se a quantidade de rodadas está correta
games_data.groupby('ano').rodata.nunique()

ano
2003    46
2004    46
2005    42
2006    38
2007    38
2008    38
2009    38
2010    38
2011    38
2012    38
2013    38
2014    38
2015    38
2016    38
2017    38
2018    38
2019    38
2020    27
2021    38
2022    38
Name: rodata, dtype: int64

Com excessão do ano de 2020, a quantidade de rodadas por ano está correta. 
Devido a isso, irei desconsiderar o ano de 2020 na análise.